In [1]:
import numpy as np 
import matplotlib.pyplot as plt 
import pandas as pd 
import os
import os.path
from scipy.optimize import curve_fit

In [2]:
# Loading file, and obtaining long name components 
path = "./testfolder"
data_list = os.listdir(path) # Create a list of all files in the folder

current_file = data_list[2]
print(f"The file being used is {current_file}.")
components = current_file.split("_")
latticeT = components[0]

The file being used is 174_p200_n1_x_100_10nm_IT_10.csv.


In [3]:
# Constants to be used 
h = 4.1357 * 10 ** -15 # Plancks constant in eV*s
c = 2.99792458 * 10 ** 8 # Speed of light in m/s 
k = 8.617333262145 * 10 ** -5 # Boltzmann constant in eV/K
comp = 0.939162 # Composition for InGaAs, estimated by Peter, solved through Wolfram Alpha 

# User input 
wl = 1550*10**-9 # in m, wavelength of LED light
T = float(latticeT) + 273.15 # Lattice temperature, K 
bandE = 0.42+0.625*comp*(5.8/(T+300)-4.19/(T+271))*10**-4 * T**2 * comp - (4.19*10**-4 * T**2)/(T+271) + 0.475*comp**2 # in eV, bandgap energy for InGaAs. 
print(f"The band gap energy is {bandE}")

The band gap energy is 0.7435616030790153


In [4]:
# Creation of dataframe
test_df = pd.read_csv(path+f"/{data_list[2]}") # For future uses, we will be doing 'for' loop for the entire directory, so there won't be a need to selectively choose the index 
col = test_df.columns # out: Index(['Wavelength', 'S2c', 'Wavelength.1', 'S2'], dtype='object')

# Creating new df to not mess with original 
new_df = pd.DataFrame() 
new_df[col[0]] = test_df[col[0]].iloc[1:] # Wavelength, using iloc to remove non numerical value 
new_df[col[1]] = test_df[col[1]].iloc[1:] # S2c 
new_df[col[3]] = test_df[col[3]].iloc[1:] # S2
new_df = new_df.astype(float) # Converting data values to numeric from strings, prep for parse in plot

# Creating additional columns to be used for further calculations 
new_df['Photon Energy'] = (h*c)/(new_df[col[0]]*10**-9)
new_df['Multiplier'] = new_df['S2c']/new_df['S2']

# dLambda conversion to dE 
scale = 1240
new_df['dE_Conv S2c'] =  new_df[col[1]] * new_df[col[0]] / scale
new_df['dE_Conv S2'] = new_df[col[3]] * new_df[col[0]] / scale

In [13]:
# Applying curve fit 

def guess_check(title_str, xdata, ydata): 
    # Curve fit function: 
    def first_pass(E, b, c): 
        return c  * E * np.sqrt(E-bandE) * np.exp(-E*b) # bandE and k are already defined previously 

    def second_pass(E, a, b, c): 
        return a + c  * E * np.sqrt(E-bandE) * np.exp(-E*b) 

    # Curve fit range and weight:
    E = np.linspace(np.min(xdata), np.max(xdata), len(xdata))
    weight_func = 1/(np.sqrt(np.exp(-E*18)))

    popt1, pcov1 = curve_fit(first_pass, xdata, ydata, maxfev = 10000, sigma = weight_func, absolute_sigma=True) 
    #print(popt1)
    p0_b = popt1[0]
    p0_c = popt1[1]

    popt2, pcov2 = curve_fit(second_pass, xdata, ydata, maxfev = 10000, p0 = [0,p0_b,p0_c], sigma=weight_func, absolute_sigma=True)
    #print(popt2)
    pfinal_a = popt2[0]
    pfinal_b = popt2[1]
    pfinal_c = popt2[2] 

    print(f''' 
{title_str}
_____________________________________________

Initial Guess Parameters: 
    b = {popt1[0]}
    c = {popt1[1]}

Final Fit Parameters: 
    a = {popt2[0]}
    b = {popt2[1]}
    c = {popt2[2]}
    ''')
    return p0_b, p0_c, pfinal_a, pfinal_b, pfinal_c 

dc_guessb, dc_guessc, dc_fita, dc_fitb, dc_fitc = guess_check('DC-Offset Calculations', new_df['Photon Energy'],new_df['S2c'])


 
DC-Offset Calculations
_____________________________________________

Initial Guess Parameters: 
    b = 23.7907522165252
    c = 270872713.8724462

Final Fit Parameters: 
    a = 0.0003610166196016698
    b = 24.128972423328143
    c = 360880435.0180705
    


In [15]:
# Multiplier Correction 
new_df['Corrected S2'] = (new_df['dE_Conv S2'] - dc_fita) * new_df['Multiplier']
guessb, guessc, final_a, final_b, final_c = guess_check('Final Fit Calculation', new_df['Photon Energy'], new_df['Corrected S2'])
e_temp = 1/(k*final_b)
print(e_temp)


 
Final Fit Calculation
_____________________________________________

Initial Guess Parameters: 
    b = 25.301711705373833
    c = 1151436053.1493118

Final Fit Parameters: 
    a = -2.2597369352413784e-05
    b = 25.283105877392597
    c = 1133449607.7637398
    
458.9830924185124
